In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tslearn
import math
import random

In [62]:
raw_points_data_filename =  "points.json"

with open(raw_points_data_filename, 'r') as f:
    json_data = json.load(f) 

p_data = json_data[1:] 

points = []
for d in p_data:
    points.extend(d['points'])

wpoints = []
for p in points:
    wpoints.append({
        "x":p['x'],
        "y":p['y'],
        "floor":p['floor'],
        "wifiData": p['wifiData']
    })

In [17]:
from tslearn.utils import to_time_series
from tslearn.metrics import dtw_path,dtw_path_from_metric

In [20]:
dtw_path_from_metric(
    s1=[[1,2,1],[1,1,2]], 
    s2=[[1,2,3]],
    global_constraint=None, # sakoe_chiba ,itakura
    sakoe_chiba_radius=None, # int
    itakura_max_slope=None, # float
    be=None, # pytorch, numpy
    metric=lambda x, y: np.sum((x-y)**2),
)

# list , similary score

([(0, 0), (1, 0)], 6.0)

In [3]:
wpoints[0]

{'x': 1326,
 'y': 30,
 'floor': 0,
 'wifiData': [{'timestamp': 301137657238,
   'level': -98,
   'frequency': 2437,
   'capabilities': '[WPA2-PSK-CCMP][RSN-PSK-CCMP][ESS][WPS]',
   'BSSID': '3a:98:b5:a4:4d:ec',
   'SSID': 'ORBI94 scharfstein'},
  {'timestamp': 301137657210,
   'level': -97,
   'frequency': 2437,
   'capabilities': '[WPA2-PSK-CCMP][RSN-PSK-CCMP][ESS][WPS]',
   'BSSID': '3a:98:b5:a3:c7:de',
   'SSID': 'ORBI94 scharfstein'},
  {'timestamp': 301137657202,
   'level': -98,
   'frequency': 2412,
   'capabilities': '[WPA2-PSK-CCMP][RSN-PSK-CCMP][ESS][WPS]',
   'BSSID': 'a2:b5:3c:b0:f4:6e',
   'SSID': 'Naama'},
  {'timestamp': 301137657255,
   'level': -80,
   'frequency': 5300,
   'capabilities': '[ESS]',
   'BSSID': '94:b3:4f:a2:ed:50',
   'SSID': 'Afeka-Wifi-Open'},
  {'timestamp': 301137657260,
   'level': -80,
   'frequency': 5300,
   'capabilities': '[ESS]',
   'BSSID': '94:b3:4f:a2:ed:54',
   'SSID': 'Afeka-Staff'},
  {'timestamp': 301133679323,
   'level': -81,
   'fre

In [ ]:

def filter_rss_by_threshold(rss_array, threshold):
    return [rss for rss in rss_array if rss >= threshold]

def linear_interpolation(rss1, rss2, n, i):
    return ((n - i) / n) * rss1 + (i / n) * rss2

def adaptive_interpolation(rss_array, f1, f2):
    n = math.floor(f1 / f2)
    interpolated_rss = []
    for i in range(len(rss_array) - 1):
        for j in range(n):
            interpolated_rss.append(linear_interpolation(rss_array[i], rss_array[i + 1], n, j))
    interpolated_rss.append(rss_array[-1])  # Include the last original RSS
    return interpolated_rss

def normalize(array):
    mean = np.mean(array)
    std_dev = np.std(array)
    return [(val - mean) / std_dev for val in array]

def calculate_weights(rss_array):
    max_rss = max(rss_array)
    return [math.exp(rss / max_rss) for rss in rss_array]

def compute_weighted_distance(rss, rssdb, weights):
    p = len(rss)
    distance = 0
    for k in range(p):
        distance += weights[k] * (rss[k] - rssdb[k]) ** 2
    return distance

def mdtw_with_wls(rss, rssdb, weights):
    n = len(rss)
    m = len(rssdb)
    d = np.zeros((n, m))

    for i in range(n):
        for j in range(m):
            d[i][j] = compute_weighted_distance(rss[i], rssdb[j], weights)

    D = np.zeros((n, m))
    D[0][0] = d[0][0]

    for i in range(1, n):
        D[i][0] = d[i][0] + D[i - 1][0]

    for j in range(1, m):
        D[0][j] = d[0][j] + D[0][j - 1]

    for i in range(1, n):
        for j in range(1, m):
            D[i][j] = d[i][j] + min(D[i - 1][j], D[i][j - 1], D[i - 1][j - 1])

    return D[n - 1][m - 1]

def weighted_least_squares(positions, distances, sigma):
    weights = [1 / math.exp(di ** 2 / (2 * sigma ** 2)) for di in distances]
    total_weight = sum(weights)

    normalized_weights = [w / total_weight for w in weights]

    estimated_x = sum(normalized_weights[i] * positions[i][0] for i in range(len(positions)))
    estimated_y = sum(normalized_weights[i] * positions[i][1] for i in range(len(positions)))

    return {'x': estimated_x, 'y': estimated_y}

def match_fingerprints(user_input, database_fingerprints):
    matched_rss = []
    matched_dbrss = []

    for user_network in user_input:
        db_network = next((db_network for db_network in database_fingerprints if db_network['ssid'] == user_network['ssid']), None)
        if db_network:
            matched_rss.append(user_network['rss'])
            matched_dbrss.append(db_network['rss'])

    return {'matched_rss': matched_rss, 'matched_dbrss': matched_dbrss}

def estimate_position(user_input, database_fingerprints, threshold, f1, f2, sigma, r):
    matched = match_fingerprints(user_input, database_fingerprints)

    filtered_rss = [filter_rss_by_threshold(rss_array, threshold) for rss_array in matched['matched_rss']]

    interpolated_rss = [adaptive_interpolation(rss_array, f1, f2) for rss_array in filtered_rss]

    normalized_rss = [normalize(rss) for rss in interpolated_rss]
    normalized_dbrss = [normalize(rss) for rss in matched['matched_dbrss']]

    weights = calculate_weights([val for sublist in interpolated_rss for val in sublist])

    distances = [mdtw_with_wls(rss, normalized_dbrss[i], weights) for i, rss in enumerate(normalized_rss)]

    sorted_distances = sorted(
        [{'distance': d, 'position': (database_fingerprints[i]['x'], database_fingerprints[i]['y'])} for i, d in enumerate(distances)],
        key=lambda x: x['distance']
    )

    top_r = sorted_distances[:r]

    positions = [item['position'] for item in top_r]
    distance_values = [item['distance'] for item in top_r]

    estimated_position = weighted_least_squares(positions, distance_values, sigma)

    return estimated_position

# Example database fingerprints with positions
database_fingerprints = [
    {
        'ssid': 'network1',
        'rss': [-78, -82, -85, -88, -74],
        'x': 5,
        'y': 10
    },
    {
        'ssid': 'network2',
        'rss': [-61, -63, -60, -55, -52],
        'x': 15,
        'y': 20
    }
]

# Example user input
user_input = [
    {
        'ssid': 'network1',
        'rss': [-80, -75, -90, -60, -50]
    },
    {
        'ssid': 'network2',
        'rss': [-62, -65, -67, -70, -69]
    }
]

# Parameters for position estimation
threshold = -70
f1 = 0.5  # Target sampling frequency
f2 = 0.125  # Original sampling frequency
sigma = 1  # Configuration parameter for exponential function
r = 2  # Number of top positions to consider

estimated_position = estimate_position(user_input, database_fingerprints, threshold, f1, f2, sigma, r)

print("Estimated Position:", estimated_position)


In [ ]:
import random

# Generate random database fingerprints
def generate_random_fingerprints(num_fingerprints, num_rss_values):
    fingerprints = []
    for _ in range(num_fingerprints):
        ssid = f"network{_ + 1}"
        rss = [random.randint(-90, -30) for _ in range(num_rss_values)]
        x = random.uniform(0, 50)
        y = random.uniform(0, 50)
        fingerprints.append({'ssid': ssid, 'rss': rss, 'x': x, 'y': y})
    return fingerprints

# Generate user inputs by slightly varying the RSS values from the database
def generate_user_inputs(database_fingerprints, num_inputs, variation):
    user_inputs = []
    for _ in range(num_inputs):
        input_entry = []
        for db_entry in random.sample(database_fingerprints, len(database_fingerprints)):
            ssid = db_entry['ssid']
            rss = [rss_val + random.randint(-variation, variation) for rss_val in db_entry['rss']]
            input_entry.append({'ssid': ssid, 'rss': rss})
        user_inputs.append(input_entry)
    return user_inputs

# Calculate accuracy based on estimated vs actual positions
def calculate_accuracy(estimated_positions, actual_positions):
    total_error = 0
    errors = []
    for est_pos, act_pos in zip(estimated_positions, actual_positions):
        error = math.sqrt((est_pos['x'] - act_pos['x']) ** 2 + (est_pos['y'] - act_pos['y']) ** 2)
        errors.append(error)
        total_error += error
    avg_error = total_error / len(estimated_positions)
    return errors, avg_error

# Generate the data
num_fingerprints = 50
num_rss_values = 5
num_inputs = 10
variation = 5

database_fingerprints = generate_random_fingerprints(num_fingerprints, num_rss_values)
user_inputs = generate_user_inputs(database_fingerprints, num_inputs, variation)

# Estimate positions for user inputs
estimated_positions = []
actual_positions = [{'x': entry['x'], 'y': entry['y']} for entry in random.sample(database_fingerprints, num_inputs)]

threshold = -70
f1 = 0.5
f2 = 0.125
sigma = 1
r = 5

for user_input in user_inputs:
    estimated_position = estimate_position(user_input, database_fingerprints, threshold, f1, f2, sigma, r)
    estimated_positions.append(estimated_position)

# Calculate and print accuracy
errors, avg_error = calculate_accuracy(estimated_positions, actual_positions)

print("Individual Errors:", errors)
print("Average Error:", avg_error)

# Example database fingerprints with positions (for reproducibility)
print("Database Fingerprints:", database_fingerprints[:5])  # Print first 5 for brevity

# Example user inputs (for reproducibility)
print("User Inputs:", user_inputs[:3])  # Print first 3 for brevity

# Estimated positions
print("Estimated Positions:", estimated_positions)


In [131]:
def print_json(data):
    print(json.dumps(data,indent=2))

def get_num_interpolation(f1,f2):
    n = math.floor(f1 / f2)
    return n

In [132]:
threshold = -70
f2 = 0.0333
f1 = 0.0333
num_interpolation = get_num_interpolation(f1,f2)

nets = ['Afeka-Staff',
'Afeka-Wifi-Open',
'Afeka-Students']

In [142]:
def filter_by_networks(point):
    point['wifiData'] = [p for p in point['wifiData'] if p['SSID'] in nets]

def filter_by_rss_threshold(point,threshold):
    point['wifiData'] = [p for p in point['wifiData'] if p['level'] >= threshold]
    
def get_rss(point):
    return [p['level'] for p in point['wifiData']]

def get_same_aps(point1,point2):
    np1 = point1.copy()
    np2 = point2.copy()
    np1['wifiData'] = []
    np2['wifiData'] = []
    for p1 in point1['wifiData']:
        for p2 in point2['wifiData']:
            if p1['SSID'] == p2['SSID'] and p1['BSSID'] == p2['BSSID']:
                np1['wifiData'].append(p1)
                np2['wifiData'].append(p2)
    return np1,np2



def linear_interpolation(rss1, rss2, n, i):
    return ((n - i) / n) * rss1 + (i / n) * rss2


def wifi_linear_interpolation(point1, point2):
    rsses = []
    for p1 in point1['wifiData']:
        found = False
        for p2 in point2['wifiData']:
            if p1['SSID'] == p2['SSID'] and p1['BSSID'] == p2['BSSID']:
                rss = {
                    "RSS": linear_interpolation(p1['level'],p2['level'], num_interpolation,len(rsses)),
                    "SSID":p1['SSID'],
                    "BSSID":p1['BSSID'],
                }
                rsses.append(rss)
            found = True
        if not found:
            raise ValueError("invalid points")
    return rsses



In [143]:
w1 = wpoints[48].copy()
w2 = wpoints[49].copy()
w3 = wpoints[50].copy()
w4 = wpoints[51].copy()


def filter_ap(point):
    a = len(point['wifiData'])
    filter_by_networks(point)
    b = len(point['wifiData'])
    filter_by_rss_threshold(point,threshold)
    c = len(point['wifiData'])
    return (a,b,c)

def info_wifi(point):
    info = {}
    for f in point['wifiData']:

        if f['SSID'] not in info:
            info[f['SSID']] = {
                'num_rss':0,
                'num_aps':0,
                'BSSIDs': {}
            }
        
        if f["BSSID"] not in info[f['SSID']]['BSSIDs']:
            info[f['SSID']]['BSSIDs'][f["BSSID"]] = []
            info[f['SSID']]['num_aps'] += 1


        info[f['SSID']]['BSSIDs'][f["BSSID"]].append(f['level'])
        info[f['SSID']]['num_rss'] += 1  

    
    strings = []
    for key,value, in info.items():
        strings.append(str(("net:",key, "num_rss",value['num_rss'], "num_aps", value['num_aps'])))
    return info,"\n".join(strings)






print("w1:",w1['x'],w1['y'],w1['floor'], filter_ap(w1))
print("w2:",w2['x'],w2['y'],w2['floor'], filter_ap(w2))
print("w3:",w3['x'],w3['y'],w3['floor'], filter_ap(w3))
print("w4:",w4['x'],w4['y'],w4['floor'], filter_ap(w4))

# print("w1:",info_wifi(w1)[1])
# print("w2:",info_wifi(w2)[1])
# print("w3:",info_wifi(w3)[1])
# print("w4:",info_wifi(w4)[1])


w_prev = w1
w_next = w2

nw11, nw12 = get_same_aps(w1,w2)

nw22, nw23 = get_same_aps(w2,w3)
nw33, nw34 = get_same_aps(w3,w4)
nw41, nw44 = get_same_aps(w1,w4)
nw51, nw53 = get_same_aps(w1,w3)

print("nw11:",info_wifi(nw11)[1])
print("nw12:",info_wifi(nw12)[1])

# print("nw22:",info_wifi(nw22)[1])
# print("nw23:",info_wifi(nw23)[1])

# print("nw33:",info_wifi(nw33)[1])
# print("nw34:",info_wifi(nw34)[1])

# print("nw41:",info_wifi(nw41)[1])
# print("nw44:",info_wifi(nw44)[1])

# print("nw51:",info_wifi(nw51)[1])
# print("nw53:",info_wifi(nw53)[1])

wifi_linear_interpolation(nw11,nw12)


w1: 622 383 0 (56, 36, 9)
w2: 682 333 0 (83, 51, 15)
w3: 627 333 0 (57, 35, 9)
w4: 557 338 0 (82, 51, 18)
nw11: ('net:', 'Afeka-Staff', 'num_rss', 2, 'num_aps', 2)
('net:', 'Afeka-Students', 'num_rss', 2, 'num_aps', 2)
('net:', 'Afeka-Wifi-Open', 'num_rss', 2, 'num_aps', 2)
nw12: ('net:', 'Afeka-Staff', 'num_rss', 2, 'num_aps', 2)
('net:', 'Afeka-Students', 'num_rss', 2, 'num_aps', 2)
('net:', 'Afeka-Wifi-Open', 'num_rss', 2, 'num_aps', 2)


[{'RSS': -36.0, 'SSID': 'Afeka-Staff', 'BSSID': '94:b3:4f:a3:a7:54'},
 {'RSS': -44.0, 'SSID': 'Afeka-Students', 'BSSID': '94:b3:4f:a3:a7:55'},
 {'RSS': -51.0, 'SSID': 'Afeka-Wifi-Open', 'BSSID': '94:b3:4f:a3:a7:50'},
 {'RSS': -58.0, 'SSID': 'Afeka-Students', 'BSSID': '94:b3:4f:a3:25:e5'},
 {'RSS': -53.0, 'SSID': 'Afeka-Staff', 'BSSID': '94:b3:4f:a3:25:e4'},
 {'RSS': -49.0, 'SSID': 'Afeka-Wifi-Open', 'BSSID': '94:b3:4f:a3:25:e0'}]

In [61]:
def MDTW():
    

In [133]:
def filter_rss_by_threshold(rss_array, threshold):
    return [rss for rss in rss_array if rss >= threshold]







In [49]:
def update_point(point,boolean_func):
    point['wifiData'] = [p for p in point['wifiData'] if boolean_func(p)]
    # def update(data,func):
    #     return {
    #         'timestamp': data['timestamp'],
    #         'level': data['level'],
    #         'frequency': data['frequency'],
    #         'capabilities': data['capabilities'],
    #         'BSSID': data['BSSID'],
    #         'SSID': data['SSID'],
    #     }
        


576 35 0
35 0 -1


9
9
9


In [80]:
w1['wifiData']

[{'timestamp': 304199259625,
  'level': -36,
  'frequency': 5260,
  'capabilities': '[ESS]',
  'BSSID': '94:b3:4f:a3:a7:54',
  'SSID': 'Afeka-Staff'},
 {'timestamp': 304199259631,
  'level': -36,
  'frequency': 5260,
  'capabilities': '[ESS]',
  'BSSID': '94:b3:4f:a3:a7:55',
  'SSID': 'Afeka-Students'},
 {'timestamp': 304199259620,
  'level': -37,
  'frequency': 5260,
  'capabilities': '[ESS]',
  'BSSID': '94:b3:4f:a3:a7:50',
  'SSID': 'Afeka-Wifi-Open'},
 {'timestamp': 304199259591,
  'level': -64,
  'frequency': 5240,
  'capabilities': '[ESS]',
  'BSSID': '94:b3:4f:a3:25:e5',
  'SSID': 'Afeka-Students'},
 {'timestamp': 304199259586,
  'level': -65,
  'frequency': 5240,
  'capabilities': '[ESS]',
  'BSSID': '94:b3:4f:a3:25:e4',
  'SSID': 'Afeka-Staff'},
 {'timestamp': 304199259576,
  'level': -64,
  'frequency': 5240,
  'capabilities': '[ESS]',
  'BSSID': '94:b3:4f:a3:25:e0',
  'SSID': 'Afeka-Wifi-Open'},
 {'timestamp': 304199259490,
  'level': -68,
  'frequency': 5220,
  'capabilitie

In [81]:
get_rss(w1)

[-36, -36, -37, -64, -65, -64, -68, -69, -69]

In [ ]:
def adaptive_interpolation(rss_array, f1, f2):
    n = get_num_interpolation(f1, f2)
    interpolated_rss = []
    for i in range(len(rss_array) - 1):
        for j in range(n):
            interpolated_rss.append(linear_interpolation(rss_array[i], rss_array[i + 1], n, j))
    interpolated_rss.append(rss_array[-1]) 
    return interpolated_rss

def normalize(array):
    mean = np.mean(array)
    std_dev = np.std(array)
    return [(val - mean) / std_dev for val in array]

def calculate_weights(rss_array):
    max_rss = max(rss_array)
    return [math.exp(rss / max_rss) for rss in rss_array]

def compute_weighted_distance(rss, rssdb, weights):
    p = len(rss)
    distance = 0
    for k in range(p):
        distance += weights[k] * (rss[k] - rssdb[k]) ** 2
    return distance

def mdtw_with_wls(rss, rssdb, weights):
    n = len(rss)
    m = len(rssdb)
    d = np.zeros((n, m))

    for i in range(n):
        for j in range(m):
            d[i][j] = compute_weighted_distance(rss[i], rssdb[j], weights)

    D = np.zeros((n, m))
    D[0][0] = d[0][0]

    for i in range(1, n):
        D[i][0] = d[i][0] + D[i - 1][0]

    for j in range(1, m):
        D[0][j] = d[0][j] + D[0][j - 1]

    for i in range(1, n):
        for j in range(1, m):
            D[i][j] = d[i][j] + min(D[i - 1][j], D[i][j - 1], D[i - 1][j - 1])

    return D[n - 1][m - 1]

def weighted_least_squares(positions, distances, sigma):
    weights = [1 / math.exp(di ** 2 / (2 * sigma ** 2)) for di in distances]
    total_weight = sum(weights)

    normalized_weights = [w / total_weight for w in weights]

    estimated_x = sum(normalized_weights[i] * positions[i][0] for i in range(len(positions)))
    estimated_y = sum(normalized_weights[i] * positions[i][1] for i in range(len(positions)))

    return {'x': estimated_x, 'y': estimated_y}

def match_fingerprints(user_input, database_fingerprints):
    matched_rss = []
    matched_dbrss = []

    for user_network in user_input:
        db_network = next((db_network for db_network in database_fingerprints if db_network['ssid'] == user_network['ssid']), None)
        if db_network:
            matched_rss.append(user_network['rss'])
            matched_dbrss.append(db_network['rss'])

    return {'matched_rss': matched_rss, 'matched_dbrss': matched_dbrss}

def estimate_position(user_input, database_fingerprints, threshold, f1, f2, sigma, r):
    matched = match_fingerprints(user_input, database_fingerprints)

    filtered_rss = [filter_rss_by_threshold(rss_array, threshold) for rss_array in matched['matched_rss']]

    interpolated_rss = [adaptive_interpolation(rss_array, f1, f2) for rss_array in filtered_rss]

    normalized_rss = [normalize(rss) for rss in interpolated_rss]
    normalized_dbrss = [normalize(rss) for rss in matched['matched_dbrss']]

    weights = calculate_weights([val for sublist in interpolated_rss for val in sublist])

    distances = [mdtw_with_wls(rss, normalized_dbrss[i], weights) for i, rss in enumerate(normalized_rss)]

    sorted_distances = sorted(
        [{'distance': d, 'position': (database_fingerprints[i]['x'], database_fingerprints[i]['y'])} for i, d in enumerate(distances)],
        key=lambda x: x['distance']
    )

    top_r = sorted_distances[:r]

    positions = [item['position'] for item in top_r]
    distance_values = [item['distance'] for item in top_r]

    estimated_position = weighted_least_squares(positions, distance_values, sigma)

    return estimated_position

# Example database fingerprints with positions
database_fingerprints = [
    {
        'ssid': 'network1',
        'rss': [-78, -82, -85, -88, -74],
        'x': 5,
        'y': 10
    },
    {
        'ssid': 'network2',
        'rss': [-61, -63, -60, -55, -52],
        'x': 15,
        'y': 20
    }
]

# Example user input
user_input = [
    {
        'ssid': 'network1',
        'rss': [-80, -75, -90, -60, -50]
    },
    {
        'ssid': 'network2',
        'rss': [-62, -65, -67, -70, -69]
    }
]

# Parameters for position estimation
threshold = -70
f1 = 0.5  # Target sampling frequency
f2 = 0.125  # Original sampling frequency
sigma = 1  # Configuration parameter for exponential function
r = 2  # Number of top positions to consider

estimated_position = estimate_position(user_input, database_fingerprints, threshold, f1, f2, sigma, r)

print("Estimated Position:", estimated_position)
